<a href="https://colab.research.google.com/github/KimJisanER/Automation_toyrent/blob/main/metabolism_Discriptor_0831_new_discriptor_sp%2Csp2%2Csp3_rd2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

import sys
sys.path

import condacolab
condacolab.check()

!mamba install -c conda-forge rdkit

✨🍰✨ Everything looks OK!
✨🍰✨ Everything looks OK!

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import rdkit
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from rdkit.Chem import Descriptors

In [3]:
def getMolDescriptors(mol, missingVal=None):
    ''' calculate the full list of descriptors for a molecule

        missingVal is used if the descriptor cannot be calculated
    '''
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res


In [4]:
# load data
train = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train.csv")
train["AlogP"] = np.where(pd.isna(train["AlogP"]), train["LogD"], train["AlogP"])

test = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/test.csv")
test["AlogP"] = np.where(pd.isna(test["AlogP"]), test["LogD"], test["AlogP"])

full = pd.concat([train, test], axis = 0).reset_index(drop=True)
full['Molecule'] = full['SMILES'].apply(Chem.MolFromSmiles)

In [5]:
full

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,Molecule
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37,<rdkit.Chem.rdchem.Mol object at 0x7d1e21f2f5a0>
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47,<rdkit.Chem.rdchem.Mol object at 0x7d1e21f2f610>
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45,<rdkit.Chem.rdchem.Mol object at 0x7d1e21f2f680>
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60,<rdkit.Chem.rdchem.Mol object at 0x7d1e21f2f6f0>
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43,<rdkit.Chem.rdchem.Mol object at 0x7d1e21f2f760>
...,...,...,...,...,...,...,...,...,...,...,...,...
3976,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,NaN,NaN,4.207,306.443,2,1,7,4.207,55.13,<rdkit.Chem.rdchem.Mol object at 0x7d1e21de10e0>
3977,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,NaN,NaN,-0.608,335.398,5,0,1,-1.736,70.16,<rdkit.Chem.rdchem.Mol object at 0x7d1e21de1150>
3978,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,NaN,NaN,1.792,349.383,3,1,3,1.792,69.72,<rdkit.Chem.rdchem.Mol object at 0x7d1e21de11c0>
3979,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,NaN,NaN,0.790,341.132,3,2,2,0.423,69.64,<rdkit.Chem.rdchem.Mol object at 0x7d1e21de1230>


In [6]:
from rdkit import DataStructs

allDescrs = [getMolDescriptors(m) for m in full['Molecule']]
full_Descrs = pd.DataFrame(allDescrs)

In [19]:
!pip install mmpdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 kB 9.5 MB/s eta 0:00:00


In [34]:
from mmpdblib import environment
def get_envsmi(constant_smi, radius):
    centers = environment.find_centers(constant_smi)
    res = []
    for atom_id in centers.atom_ids:
        env = Chem.FindAtomEnvironmentOfRadiusN(centers.mol, radius, atom_id)
        submol = Chem.PathToSubmol(centers.mol, env)
        smi = Chem.MolToSmiles(submol)
        res.append(smi)
    return ".".join(res)

In [93]:
from rdkit import Chem
import pandas as pd

def extract_sp2_carbon_fragments_rd2(smiles, radius):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP2 and atom.IsInRing() == False:
            env = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, atom.GetIdx())
            submol = Chem.PathToSubmol(mol, env)
            fragments.append(submol)

    return fragments

radius = 2
fragment_counts = {}

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_sp2_carbon_fragments_rd2(smiles, radius)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df_rd2 = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
result_df_rd2.columns = ['rd2_sp2_' + column for column in result_df_rd2.columns]

# 0이 아닌 값의 수가 100 미만인 컬럼 제거
min_non_zero_count = 30
columns_to_keep = []

for column in result_df_rd2.columns:
    non_zero_count = (result_df_rd2[column] != 0).sum()
    if non_zero_count >= min_non_zero_count:
        columns_to_keep.append(column)

result_df_rd2_filtered = result_df_rd2[columns_to_keep]

# 결과 데이터프레임 출력
print(result_df_rd2_filtered)


      rd2_sp2_cc(c)OCC  rd2_sp2_cCNC(c)=O  rd2_sp2_cC(N)=O  rd2_sp2_CC(N)=O  \
0                  2.0                1.0              1.0              0.0   
1                  0.0                0.0              0.0              1.0   
2                  0.0                0.0              0.0              0.0   
3                  0.0                0.0              0.0              0.0   
4                  0.0                0.0              1.0              0.0   
...                ...                ...              ...              ...   
3976               0.0                0.0              0.0              1.0   
3977               0.0                0.0              0.0              2.0   
3978               0.0                0.0              2.0              1.0   
3979               0.0                0.0              1.0              0.0   
3980               0.0                0.0              0.0              0.0   

      rd2_sp2_cS(N)(=O)=O  rd2_sp2_cc(c)OC  rd2_sp2

In [94]:
from rdkit import Chem
import pandas as pd

def extract_sp3_carbon_fragments_rd2(smiles, radius):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP3 and atom.IsInRing() == False:
            env = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, atom.GetIdx())
            submol = Chem.PathToSubmol(mol, env)
            fragments.append(submol)

    return fragments

radius = 2
fragment_counts = {}

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_sp3_carbon_fragments_rd2(smiles, radius)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df_rd2_sp3 = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
result_df_rd2_sp3.columns = ['rd2_sp3_' + column for column in result_df_rd2_sp3.columns]

# 0이 아닌 값의 수가 100 미만인 컬럼 제거
min_non_zero_count = 30
columns_to_keep = []

for column in result_df_rd2_sp3.columns:
    non_zero_count = (result_df_rd2_sp3[column] != 0).sum()
    if non_zero_count >= min_non_zero_count:
        columns_to_keep.append(column)

result_df_rd2_sp3_filtered = result_df_rd2_sp3[columns_to_keep]

# 결과 데이터프레임 출력
print(result_df_rd2_sp3_filtered)


      rd2_sp3_CCO  rd2_sp3_CCOc  rd2_sp3_cc(c)CNC  rd2_sp3_cc(C)n  \
0             2.0           2.0               1.0             1.0   
1             0.0           0.0               0.0             1.0   
2             0.0           0.0               0.0             0.0   
3             0.0           0.0               0.0             2.0   
4             0.0           0.0               0.0             0.0   
...           ...           ...               ...             ...   
3976          0.0           0.0               0.0             0.0   
3977          0.0           0.0               0.0             0.0   
3978          0.0           0.0               0.0             0.0   
3979          0.0           0.0               0.0             0.0   
3980          0.0           0.0               0.0             0.0   

      rd2_sp3_CC(C)C  rd2_sp3_CCC  rd2_sp3_CCCN  rd2_sp3_CCCN(C)C  \
0                0.0          0.0           0.0               0.0   
1                1.0          0.0

In [110]:
from rdkit import Chem
import pandas as pd

def extract_sp_carbon_fragments_rd2(smiles, radius):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP and atom.IsInRing() == False:
            env = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, atom.GetIdx())
            submol = Chem.PathToSubmol(mol, env)
            fragments.append(submol)

    return fragments

radius = 2
fragment_counts = {}

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_sp_carbon_fragments_rd2(smiles, radius)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df_rd2_sp = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
result_df_rd2_sp.columns = ['rd2_sp_' + column for column in result_df_rd2_sp.columns]

# 0이 아닌 값의 수가 100 미만인 컬럼 제거
min_non_zero_count = 30
columns_to_keep = []

for column in result_df_rd2_sp.columns:
    non_zero_count = (result_df_rd2_sp[column] != 0).sum()
    if non_zero_count >= min_non_zero_count:
        columns_to_keep.append(column)

result_df_rd2_sp_filtered = result_df_rd2_sp[columns_to_keep]

# 결과 데이터프레임 출력
print(result_df_rd2_sp_filtered)


      rd2_sp_cc(c)C#N  rd2_sp_cC#N  rd2_sp_cc(n)C#N
0                 0.0          0.0              0.0
1                 0.0          0.0              0.0
2                 0.0          0.0              0.0
3                 0.0          0.0              0.0
4                 0.0          0.0              0.0
...               ...          ...              ...
3976              0.0          0.0              0.0
3977              0.0          0.0              0.0
3978              0.0          0.0              0.0
3979              0.0          0.0              0.0
3980              0.0          0.0              0.0

[3981 rows x 3 columns]


In [64]:
fragment_counts = {}

def extract_sp3_carbon_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP3 and atom.IsInRing() == False:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_sp3_carbon_fragments(smiles)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')

# 결과 데이터프레임 출력
print(result_df)

       CC  CCO  cCN   Cc  CCC  CCN  cS(N)(=O)=O   CO   cF   CN  ...  CBr  \
0     2.0  2.0  1.0  2.0  0.0  0.0          0.0  0.0  0.0  0.0  ...  0.0   
1     1.0  0.0  1.0  2.0  0.0  0.0          0.0  0.0  0.0  0.0  ...  0.0   
2     1.0  0.0  0.0  0.0  1.0  1.0          0.0  0.0  0.0  0.0  ...  0.0   
3     0.0  0.0  0.0  3.0  0.0  0.0          1.0  0.0  0.0  0.0  ...  0.0   
4     1.0  0.0  0.0  1.0  0.0  0.0          0.0  0.0  0.0  0.0  ...  0.0   
...   ...  ...  ...  ...  ...  ...          ...  ...  ...  ...  ...  ...   
3976  3.0  0.0  0.0  0.0  1.0  1.0          0.0  0.0  0.0  0.0  ...  0.0   
3977  2.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  1.0  ...  0.0   
3978  2.0  0.0  0.0  0.0  0.0  1.0          0.0  0.0  0.0  0.0  ...  0.0   
3979  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  1.0  0.0  ...  0.0   
3980  2.0  0.0  0.0  0.0  1.0  1.0          0.0  0.0  0.0  1.0  ...  0.0   

      C[C@@H](C)N  C[C@H](C)O  CS(N)=O  CC(c)n  CC(O)O  CC(C)F  COP  \
0             0.

In [65]:
fragment_counts = {}

def extract_non_ring_sp2_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP2 and atom.IsInRing() == False:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_non_ring_sp2_fragments(smiles)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df2 = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
result_df2.columns = ['non_ring_sp2_' + column for column in result_df2.columns]

# 결과 데이터프레임 출력
print(result_df2)


      non_ring_sp2_COc  non_ring_sp2_CNC  non_ring_sp2_cC(N)=O  \
0                  1.0               1.0                   1.0   
1                  0.0               0.0                   0.0   
2                  0.0               0.0                   0.0   
3                  0.0               0.0                   0.0   
4                  0.0               0.0                   1.0   
...                ...               ...                   ...   
3976               0.0               1.0                   0.0   
3977               0.0               0.0                   0.0   
3978               0.0               1.0                   0.0   
3979               0.0               0.0                   1.0   
3980               0.0               0.0                   0.0   

      non_ring_sp2_C=O  non_ring_sp2_cOC  non_ring_sp2_O=S  non_ring_sp2_c=O  \
0                  1.0               1.0               0.0               0.0   
1                  1.0               0.0       

In [66]:
fragment_counts = {}

def extract_sp_carbon_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_sp_carbon_fragments(smiles)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df3 = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
result_df3.columns = ['sp_' + column for column in result_df3.columns]

# 결과 데이터프레임 출력
print(result_df3)

      sp_cC#N  sp_C#N  sp_CC#N  sp_C#CN  sp_C#Cc  sp_C#Cn  sp_N#CN  sp_C#CC
0         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
1         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
2         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
3         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
4         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
...       ...     ...      ...      ...      ...      ...      ...      ...
3976      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
3977      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
3978      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
3979      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0
3980      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0

[3981 rows x 8 columns]


In [67]:
fragment_counts = {}

def extract_ring_sp2_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP2 and atom.IsInRing() == True:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_ring_sp2_fragments(smiles)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df4 = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
result_df4.columns = ['ring_sp2_' + column for column in result_df4.columns]

# 결과 데이터프레임 출력
print(result_df4)


      ring_sp2_cc(c)O  ring_sp2_ccc  ring_sp2_cc(c)C  ring_sp2_cc(C)[nH]  \
0                 2.0           4.0              1.0                 1.0   
1                 0.0           4.0              0.0                 0.0   
2                 0.0           4.0              0.0                 0.0   
3                 0.0           6.0              1.0                 0.0   
4                 1.0           5.0              2.0                 0.0   
...               ...           ...              ...                 ...   
3976              0.0           0.0              1.0                 0.0   
3977              0.0           0.0              0.0                 0.0   
3978              0.0           8.0              3.0                 0.0   
3979              1.0           3.0              1.0                 0.0   
3980              0.0           2.0              0.0                 0.0   

      ring_sp2_cc(-c)n  ring_sp2_cc(-c)s  ring_sp2_csc  ring_sp2_Cc(n)s  \
0           

In [68]:
fragment_counts = {}

def extract_sp3_ring_carbon_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP3 and atom.IsInRing() == True:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_sp3_ring_carbon_fragments(smiles)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df5 = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
result_df5.columns = ['ring_sp3_' + column for column in result_df5.columns]

# 결과 데이터프레임 출력
print(result_df5)

      ring_sp3_CCN  ring_sp3_CC(C)C  ring_sp3_CN(C)C  ring_sp3_CN(C)S  \
0              0.0              0.0              0.0              0.0   
1              1.0              1.0              0.0              0.0   
2              4.0              0.0              1.0              0.0   
3              4.0              0.0              1.0              1.0   
4              1.0              0.0              0.0              0.0   
...            ...              ...              ...              ...   
3976           0.0              0.0              0.0              0.0   
3977           6.0              0.0              1.0              0.0   
3978           0.0              0.0              0.0              0.0   
3979           0.0              0.0              0.0              0.0   
3980           4.0              0.0              2.0              0.0   

      ring_sp3_CC(C)N  ring_sp3_CCC  ring_sp3_CC(C)O  ring_sp3_CC(C)n  \
0                 0.0           0.0              0

In [111]:
Data = pd.concat([full, full_Descrs, result_df, result_df_rd2_filtered, result_df_rd2_sp3_filtered, result_df_rd2_sp_filtered, result_df2, result_df3, result_df4, result_df5], axis=1).drop(columns=['AlogP', 'Molecule', 'MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])

In [112]:
Data['beyond_Lipinski'] = 0

Data.loc[Data['Molecular_Weight'] >= 500, 'beyond_Lipinski'] += 1
Data.loc[Data['LogD'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_H_Donors'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_H_Acceptors'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_RotatableBonds'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[(Data['Num_H_Donors'] + Data['Num_H_Acceptors']) >= 12, 'beyond_Lipinski'] += 1
Data.loc[Data['TPSA'] >= 140, 'beyond_Lipinski'] += 1
Data.loc[Data['FractionCSP3'] >= 0.3, 'beyond_Lipinski'] += 1
Data.loc[Data['NumAromaticRings'] >= 5, 'beyond_Lipinski'] += 1

In [113]:
Data['Lipinski'] = 0

Data.loc[Data['Molecular_Weight'] >= 500, 'Lipinski'] += 1
Data.loc[Data['LogD'] >= 5, 'Lipinski'] += 1
Data.loc[Data['Num_H_Donors'] >= 5, 'Lipinski'] += 1
Data.loc[Data['Num_H_Acceptors'] >= 10, 'Lipinski'] += 1

In [114]:
df_grouped = Data.groupby('SMILES').agg({'MLM': 'mean', 'HLM': 'mean'}).reset_index()

In [115]:
df_grouped

,SMILES,MLM,HLM
0,Brc1c[nH]nc1-c1nc2c3ccccc3ncn2n1,99.000,99.000
1,Brc1cc(-c2noc([C@H]3CCCO3)n2)c2ncnn2c1,3.780,35.920
2,Brc1ccc(-c2c[nH]c(CNc3nncn3-c3ccccc3)n2)cc1,8.989,50.893
3,Brc1ccc(-c2n[nH]c(-c3ccccc3)c2-c2ncc[nH]2)cc1,39.870,85.910
4,Brc1ccc(C2CC(c3ccc(Br)cc3)n3ncnc3N2)cc1,42.647,59.504
...,...,...,...
3949,c1cnc2c(c1)OC(Cn1nnc3ncc(-c4ccc(OCC5CCNCC5)cc4...,86.566,91.896
3950,c1csc(COc2ccc(CNn3cnnc3)cc2)c1,56.030,84.800
3951,c1nc(N2CCCC2)sc1CN1CC[C@H]2OCc3cnnn3[C@H]2C1,80.400,99.050
3952,c1nc(N2CCN(C3CCCCC3)CC2)c2nc[nH]c2n1,25.370,97.100


In [116]:
missing_indices = train[train.isnull().any(axis=1)].index

train = Data.iloc[:3498].dropna(axis=0).dropna(axis=1)
test = Data.iloc[3498:].drop(columns=missing_indices)

In [117]:
train

,id,SMILES,MLM,HLM,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,...,ring_sp3_cC(c)(C)O,ring_sp3_C[C@H](C)n,ring_sp3_cC(c)n,ring_sp3_CC(n)O,ring_sp3_cC(c)S,ring_sp3_cC(N)n,ring_sp3_cC(c)(c)O,ring_sp3_NS(N)(=O)=O,beyond_Lipinski,Lipinski
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,400.495,5,2,8,3.259,117.37,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,301.407,2,1,2,2.172,73.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,297.358,5,0,3,1.585,62.45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,494.652,6,0,5,3.475,92.60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,268.310,3,0,1,2.337,42.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,396.195,3,1,5,3.409,64.74,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,359.381,4,1,3,1.844,77.37,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,261.320,3,1,6,2.124,70.14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,284.696,5,1,5,0.989,91.51,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [118]:
Data_new = train.drop_duplicates(subset='SMILES').drop(columns=['MLM','HLM'])

In [119]:
train = pd.merge(Data_new, df_grouped, on='SMILES', how='left')

In [120]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3469 entries, 0 to 3468
Columns: 1011 entries, id to HLM
dtypes: float64(903), int64(106), object(2)
memory usage: 26.8+ MB


In [121]:
#결측 확인
for col in train.columns:
    has_missing_values = train[col].isna().any()
    count =  train[col].isna().sum()
    if has_missing_values:
        print(f"Column '{col}' has missing values.")
        print(f"'{count}'")

In [122]:
for col in test.columns:
    has_missing_values = test[col].isna().any()
    count =  test[col].isna().sum()
    if has_missing_values:
        print(f"Column '{col}' has missing values.")
        print(f"'{count}'")

Column 'MLM' has missing values.
'483'
Column 'HLM' has missing values.
'483'


In [123]:
# 숫자형식 컬럼들의 min-max 정규화
scaler = MinMaxScaler()
non_features = ['id', 'SMILES', 'MLM', 'HLM','Fingerprint']
features = [column for column in train.columns if column not in non_features]
train[features] = scaler.fit_transform(train[features])

test[features] = scaler.transform(test[features])

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasatt

In [124]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error
import math
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Define features and targets
non_features = ['id', 'SMILES', 'MLM', 'HLM', 'Fingerprint']
features = [column for column in train.columns if column not in non_features]
mlm_target = "MLM"
hlm_target = "HLM"

# Initialize KFold
seed = 42
n_splits = 10
kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)

# Initialize arrays to store models and scores
reg_mlms = []
reg_hlms = []

# Initialize arrays to store RMSE scores
mlm_rmse_scores = []
hlm_rmse_scores = []

# Loop through KFold splits
for i, (train_index, valid_index) in enumerate(kf.split(train)):
    df_train = train.iloc[train_index]
    df_valid = train.iloc[valid_index]

    x_train_num = df_train[features].values

    y_mlm_train = df_train[mlm_target].values
    y_hlm_train = df_train[hlm_target].values

    x_valid_num = df_valid[features].values

    y_mlm_valid = df_valid[mlm_target].values
    y_hlm_valid = df_valid[hlm_target].values

    # 이미지 입력 경로
    x1_input = keras.Input(shape=(x_train_num.shape[1],))
    x1 = layers.Dense(256, activation='relu')(x1_input)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    merged = layers.Dense(1024, activation='relu')(x1)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(512, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(128, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)

    # 회귀 모델 출력 레이어 추가
    outputs = layers.Dense(1)(merged)

    # 모델 생성
    model = keras.Model(inputs=x1_input, outputs=outputs)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=8,
    decay_rate=0.98)
    optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])

    # Create and compile another model for HLM
    # 이미지 입력 경로
    x1_input = keras.Input(shape=(x_train_num.shape[1],))
    x1 = layers.Dense(256, activation='relu')(x1_input)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    merged = layers.Dense(1024, activation='relu')(x1)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(512, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(128, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)

    # 회귀 모델 출력 레이어 추가
    outputs = layers.Dense(1)(merged)

    # 모델 생성
    model_hlm = keras.Model(inputs=x1_input, outputs=outputs)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=8,
    decay_rate=0.98)
    optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

    model_hlm.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])


    # Train the model
    checkpoint_mlm = ModelCheckpoint(f'model_mlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_mlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model.fit(x_train_num, y_mlm_train, epochs=500, batch_size=16, verbose=1, validation_data=(x_valid_num, y_mlm_valid),
              callbacks=[checkpoint_mlm, early_stopping_mlm])

    reg_mlms.append(model)

    checkpoint_hlm = ModelCheckpoint(f'model_hlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_hlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model_hlm.fit(x_train_num, y_hlm_train, epochs=500, batch_size=16, verbose=1, validation_data=(x_valid_num, y_hlm_valid),
                  callbacks=[checkpoint_hlm, early_stopping_hlm])

    reg_hlms.append(model_hlm)

    # Calculate RMSE for MLM predictions
    y_mlm_pred = model.predict(x_valid_num)
    mlm_rmse = math.sqrt(mean_squared_error(y_mlm_valid, y_mlm_pred))
    print(mlm_rmse)
    mlm_rmse_scores.append(mlm_rmse)

    # Calculate RMSE for HLM predictions
    y_hlm_pred = model_hlm.predict(x_valid_num)
    hlm_rmse = math.sqrt(mean_squared_error(y_hlm_valid, y_hlm_pred))
    print(hlm_rmse)
    hlm_rmse_scores.append(hlm_rmse)

Epoch 1/500
196/196 [==============================] - 3s 7ms/step - loss: 1299.3756 - root_mean_squared_error: 36.0469 - val_loss: 43748088.0000 - val_root_mean_squared_error: 6614.2334
Epoch 2/500
196/196 [==============================] - 1s 5ms/step - loss: 1036.2948 - root_mean_squared_error: 32.1915 - val_loss: 1044.0668 - val_root_mean_squared_error: 32.3120
Epoch 3/500
196/196 [==============================] - 1s 5ms/step - loss: 929.4553 - root_mean_squared_error: 30.4870 - val_loss: 1026.1914 - val_root_mean_squared_error: 32.0342
Epoch 4/500
196/196 [==============================] - 1s 5ms/step - loss: 905.5804 - root_mean_squared_error: 30.0929 - val_loss: 966.5415 - val_root_mean_squared_error: 31.0893
Epoch 5/500
196/196 [==============================] - 1s 5ms/step - loss: 896.6869 - root_mean_squared_error: 29.9447 - val_loss: 972.8212 - val_root_mean_squared_error: 31.1901
Epoch 6/500
196/196 [==============================] - 1s 5ms/step - loss: 863.3348 - root_mea

In [125]:
from keras.models import load_model

# Load MLM models
mlm_models = [load_model(f'model_mlm_fold{i}.h5') for i in range(n_splits)]

# Load HLM models
hlm_models = [load_model(f'model_hlm_fold{i}.h5') for i in range(n_splits)]

x_test = test[features].values

mlm_predictions = []
hlm_predictions = []

for model_mlm, model_hlm in zip(mlm_models, hlm_models):
    mlm_predictions.append(model_mlm.predict(x_test))
    hlm_predictions.append(model_hlm.predict(x_test))

# Convert prediction lists to numpy arrays
mlm_predictions = np.array(mlm_predictions)
hlm_predictions = np.array(hlm_predictions)

mlm_ensemble_prediction = mlm_predictions.mean(axis=0)
hlm_ensemble_prediction = hlm_predictions.mean(axis=0)

16/16 [==============================] - 0s 2ms/step


In [126]:
df_submission = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/sample_submission.csv")
df_submission["MLM"] = mlm_ensemble_prediction
df_submission["HLM"] = hlm_ensemble_prediction
df_submission.to_csv("submission.csv", index = False, encoding = "utf-8-sig")

In [127]:
df_submission

,id,MLM,HLM
0,TEST_000,16.352674,36.271130
1,TEST_001,72.543480,85.055565
2,TEST_002,42.092110,54.693291
3,TEST_003,52.001930,76.525139
4,TEST_004,41.343513,65.933556
...,...,...,...
478,TEST_478,11.633577,24.351810
479,TEST_479,77.857277,90.430786
480,TEST_480,32.084915,42.677334
481,TEST_481,46.583599,66.882645
